In [ ]:
import pandas as pd

In [ ]:
# List of categorical variables
categorical_variables = [
    "OriginCountryCode", "DestinationCountryCode", "RouteCode", "ShipmentStatus",
    "SalesMarket", "Dominant Mot", "ProductLine", "BusinessUnit",
    "CarriageType", "LegMode"
]

# List of numerical variables
numerical_variables = [
    "FreightWeight", "Distances_km", "co2_equivalent_t_ttw", 
    "co2_equivalent_t_wtw", "co2e_p_tkm_grammepertonnekilometers_ttw", 
    "co2e_p_tkm_grammepertonnekilometers_wtw"
]

In [ ]:
def create_bar_chart(data, variable):
    value_counts = data[variable].value_counts()
    plt.figure(figsize=(8, 6))
    value_counts.plot(kind='bar')
    plt.title(f"Diagramme en barres de {variable}")
    plt.xlabel(variable)
    plt.ylabel("Fréquence")
    plt.tight_layout()
    plt.show()

In [ ]:
#  Creates a bar chart for the top N (5 ou 10) most frequent modalities of a specified variable.
def create_top_modalities_bar_chart(data, variable, top_n):
    value_counts = data[variable].value_counts().head(top_n)
    plt.figure(figsize=(8, 6))
    value_counts.plot(kind='bar', color='skyblue')
    plt.title(f"Diagramme en barres des {top_n} modalités les plus fréquentes de {variable}")
    plt.xlabel(variable)
    plt.ylabel("Fréquence")
    plt.tight_layout()
    plt.show()

In [ ]:
def create_histogram(data, variable, bins=20):
    plt.figure(figsize=(8, 6))
    plt.hist(data[variable].dropna(), bins=bins, color='skyblue', edgecolor='black')
    plt.title(f"Histogramme de {variable}")
    plt.xlabel(variable)
    plt.ylabel("Fréquence")
    plt.tight_layout()
    plt.show()


In [ ]:
# Répartition des émissions par mode de transport et pays.

import matplotlib.pyplot as plt

# 1. Emissions by Mode of Transport and Country
emissions_by_country_mode = df.groupby(["OriginCountryCode", "Dominant Mot"]).agg({
    "co2_equivalent_t_ttw": "sum",
    "co2_equivalent_t_wtw": "sum"
}).reset_index()

plt.figure(figsize=(12, 8))
sns.barplot(
    data=emissions_by_country_mode,
    x="OriginCountryCode",
    y="co2_equivalent_t_wtw",
    hue="Dominant Mot"
)
plt.title("Emissions par mode de transport et pays (WTW)")
plt.xlabel("Pays")
plt.ylabel("Emissions de CO2 totale (WTW, tonnes)")
plt.legend(title="Mode de transport")
plt.tight_layout()
plt.show()


In [ ]:
# Emissions by Product Line and Mode of Transport
emissions_by_product_mode = df.groupby(["ProductLine", "Dominant Mot"]).agg({
    "co2_equivalent_t_ttw": "sum",
    "co2_equivalent_t_wtw": "sum"
}).reset_index()

plt.figure(figsize=(12, 8))
sns.barplot(
    data=emissions_by_product_mode,
    x="ProductLine",
    y="co2_equivalent_t_wtw",
    hue="Dominant Mot",
    palette="viridis"
)
plt.title("Emissions par Product Line et Mode of Transport (WTW)")
plt.xlabel("Product Line")
plt.ylabel("Emissions de CO2 totale (WTW, tonnes)")
plt.legend(title="Mode of Transport")
plt.tight_layout()
plt.show()

# Checking data quality: missing values, anomalies, and consistency

In [ ]:
# 1. Check for missing values
missing_values = df.isnull().sum()

# Checking for rows with missing values
rows_with_missing_data = df[df.isnull().any(axis=1)]

# Analyzing the percentage of missing data per column
missing_percentage = df.isnull().mean() * 100


# 2. Check for anomalies (e.g., negative emissions)
negative_values = {
    "co2_equivalent_t_ttw": (df["co2_equivalent_t_ttw"] < 0).sum(),
    "co2_equivalent_t_wtw": (df["co2_equivalent_t_wtw"] < 0).sum(),
    "co2e_p_tkm_grammepertonnekilometers_ttw": (df["co2e_p_tkm_grammepertonnekilometers_ttw"] < 0).sum(),
    "co2e_p_tkm_grammepertonnekilometers_wtw": (df["co2e_p_tkm_grammepertonnekilometers_wtw"] < 0).sum(),
    "FreightWeight": (df["FreightWeight"] < 0).sum(),
    "Distances_km": (df["Distances_km"] < 0).sum(),
}

# 3. Check consistency between related variables
# For instance, "co2_equivalent_t_ttw" should generally be less than "co2_equivalent_t_wtw"
consistency_check = (df["co2_equivalent_t_ttw"] > df["co2_equivalent_t_wtw"]).sum()

# Summarizing results
data_quality_summary = {
    "Valeurs Manquantes": missing_values[missing_values > 0],
    "Comptes de Valeurs Négatives": negative_values,
    "Relations Incohérentes": {"co2_ttw > co2_wtw": consistency_check},
}


# Segmentation par pays pour identifier les spécificités locales (mix énergétique, infrastructures, etc.).

In [ ]:
# Grouping data by country for analysis
country_segmentation = df.groupby("OriginCountryCode").agg({
    "Distances_km": ["mean", "median", "sum"],
    "FreightWeight": ["mean", "sum"],
    "co2_equivalent_t_ttw": ["mean", "sum"],
    "co2_equivalent_t_wtw": ["mean", "sum"],
    "ShipmentId": "count"  # Number of shipments per country
}).reset_index()

country_segmentation.columns = [
    "OriginCountryCode",
    "Avg_Distance_km",
    "Median_Distance_km",
    "Total_Distance_km",
    "Avg_FreightWeight",
    "Total_FreightWeight",
    "Avg_CO2_TTW",
    "Total_CO2_TTW",
    "Avg_CO2_WTW",
    "Total_CO2_WTW",
    "ShipmentCount"
]

In [ ]:
# Selecting a country with high emissions for detailed analysis
selected_country = country_segmentation.sort_values(by="Total_CO2_WTW", ascending=False).iloc[0]["OriginCountryCode"]
country_data = df_clean[df_clean["OriginCountryCode"] == selected_country]

# a) Breakdown by Transportation Mode for the selected country
transport_mode_country = country_data.groupby(["DominantMot", "LegMode"]).agg({
    "co2_equivalent_t_wtw": "sum"
}).reset_index()

plt.figure(figsize=(12, 8))
sns.barplot(
    data=transport_mode_country,
    x="DominantMot",
    y="co2_equivalent_t_wtw",
    hue="LegMode",
    palette="viridis"
)
plt.title(f"Emissions Breakdown by Transportation Mode for {selected_country} (WTW)")
plt.xlabel("Dominant Mode of Transport")
plt.ylabel("Total Emissions (WTW, tons)")
plt.legend(title="Leg Mode")
plt.tight_layout()
plt.show()

# b) Geographical Analysis for the selected country
geographical_emissions_country = country_data.groupby("DestinationCountryCode").agg({
    "co2_equivalent_t_wtw": "sum"
}).reset_index().sort_values(by="co2_equivalent_t_wtw", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(
    data=geographical_emissions_country,
    x="DestinationCountryCode",
    y="co2_equivalent_t_wtw",
    palette="coolwarm"
)
plt.title(f"Emissions by Destination for {selected_country} (WTW)")
plt.xlabel("Destination Country")
plt.ylabel("Total Emissions (WTW, tons)")
plt.tight_layout()
plt.show()


# Apres on peut faire des graphiques sur Dominant Mot ProductLine BusinessUnit Carrier pour le pays selectionné pour examiner les caracteristiques
# specifiques d'un pays

In [ ]:
# Analyze top CO2-emitting product lines 
# Grouping data by ProductLine for CO2 analysis 
product_line_analysis = df.groupby("ProductLine").agg({ 
"co2_equivalent_t_ttw": "sum", 
"co2_equivalent_t_wtw": "sum", 
"ShipmentId": "count" }).reset_index()

## Concentrer sur un Product Line (ici 1 par ex)

In [ ]:
# Filtering data for Product Line 1 (PL_1)
pl_1_data = df[df["ProductLine"] == "PL_1"]

# a) Breakdown by Transportation Mode
transport_mode_breakdown = pl_1_data.groupby(["DominantMot", "LegMode"]).agg({
    "co2_equivalent_t_wtw": "sum"
}).reset_index()

# Plotting transportation mode contribution to emissions
plt.figure(figsize=(12, 8))
sns.barplot(
    data=transport_mode_breakdown,
    x="DominantMot",
    y="co2_equivalent_t_wtw",
    hue="LegMode",
    palette="viridis"
)
plt.title("Emissions Breakdown by Transportation Mode for PL_1 (WTW)")
plt.xlabel("Dominant Mode of Transport")
plt.ylabel("Total Emissions (WTW, tons)")
plt.legend(title="Leg Mode")
plt.tight_layout()
plt.show()

# b) Geographical Analysis
geographical_emissions = pl_1_data.groupby(["OriginCountryCode", "DestinationCountryCode"]).agg({
    "co2_equivalent_t_wtw": "sum"
}).reset_index().sort_values(by="co2_equivalent_t_wtw", ascending=False)

# Top routes with the highest emissions
top_routes = geographical_emissions.head(10)

# Plotting geographical emissions
plt.figure(figsize=(10, 6))
sns.barplot(
    data=top_routes,
    y="co2_equivalent_t_wtw",
    x="OriginCountryCode",
    hue="DestinationCountryCode",
    palette="coolwarm"
)
plt.title("Top Routes for PL_1 with Highest Emissions (WTW)")
plt.xlabel("Origin Country")
plt.ylabel("Total Emissions (WTW, tons)")
plt.legend(title="Destination Country", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()